In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os

In [2]:
DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/cifar_data'

In [3]:
os.makedirs(DATA_DIR, exist_ok=True)

In [4]:
cifar10_path = os.path.join(DATA_DIR, 'cifar-10-batches-py')

In [5]:
if os.path.exists(cifar10_path):
    print(f"✓ 기존 데이터 발견: {cifar10_path}")
    print("저장된 데이터를 로드합니다...")
else:
    print(f"데이터가 없습니다. {DATA_DIR}에 다운로드합니다...")


데이터가 없습니다. /content/drive/MyDrive/Colab Notebooks/cifar_data에 다운로드합니다...


In [6]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print(f"✓ 데이터 로드 완료")
print(f"  - 학습 데이터: {x_train.shape}")
print(f"  - 테스트 데이터: {x_test.shape}\n")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step
✓ 데이터 로드 완료
  - 학습 데이터: (50000, 32, 32, 3)
  - 테스트 데이터: (10000, 32, 32, 3)



In [7]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [8]:
selected_classes = ['cat','dog','horse']
selected_idx = [class_names.index(c)for c in selected_classes]

In [9]:
train_mask = np.isin(y_train, selected_idx).flatten()
test_mask = np.isin(y_test, selected_idx).flatten()

x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test, y_test = x_test[test_mask], y_test[test_mask]

print(f"선택된 클래스: {selected_classes}")
print(f"  - 학습 샘플 수: {len(x_train)}")
print(f"  - 테스트 샘플 수: {len(x_test)}\n")

선택된 클래스: ['cat', 'dog', 'horse']
  - 학습 샘플 수: 15000
  - 테스트 샘플 수: 3000



In [12]:
label_map = {v: i for i, v in enumerate(selected_idx)}
y_train = np.array([label_map[int(y)]for y in y_train])
y_test = np.array([label_map[int(y)]for y in y_test])

/tmp/ipython-input-2144340769.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_train = np.array([label_map[int(y)]for y in y_train])
/tmp/ipython-input-2144340769.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_test = np.array([label_map[int(y)]for y in y_test])


In [13]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [14]:
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

x_train shape: (15000, 32, 32, 3)
x_test shape: (3000, 32, 32, 3)


In [15]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [24]:
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.1,
    verbose=2
)

Epoch 1/10
106/106 - 17s - 159ms/step - accuracy: 0.9611 - loss: 0.1012 - val_accuracy: 0.7613 - val_loss: 0.9200
Epoch 2/10
106/106 - 17s - 165ms/step - accuracy: 0.9650 - loss: 0.0982 - val_accuracy: 0.7667 - val_loss: 0.9572
Epoch 3/10
106/106 - 21s - 197ms/step - accuracy: 0.9622 - loss: 0.1027 - val_accuracy: 0.7713 - val_loss: 0.8906
Epoch 4/10
106/106 - 17s - 158ms/step - accuracy: 0.9630 - loss: 0.0987 - val_accuracy: 0.7607 - val_loss: 0.9455
Epoch 5/10
106/106 - 17s - 157ms/step - accuracy: 0.9655 - loss: 0.0920 - val_accuracy: 0.7600 - val_loss: 0.9317
Epoch 6/10
106/106 - 21s - 195ms/step - accuracy: 0.9620 - loss: 0.1026 - val_accuracy: 0.7707 - val_loss: 0.8998
Epoch 7/10
106/106 - 17s - 157ms/step - accuracy: 0.9661 - loss: 0.0893 - val_accuracy: 0.7667 - val_loss: 0.9890
Epoch 8/10
106/106 - 22s - 207ms/step - accuracy: 0.9712 - loss: 0.0799 - val_accuracy: 0.7640 - val_loss: 0.9813
Epoch 9/10
106/106 - 19s - 179ms/step - accuracy: 0.9655 - loss: 0.0892 - val_accuracy: 

In [25]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

94/94 - 1s - 12ms/step - accuracy: 0.7753 - loss: 0.9316

✅ Test Accuracy: 0.7753


In [26]:
predictions = model.predict(x_test[:10])
print("예측 결과:", np.argmax(predictions, axis=1))
print("실제 레이블:", y_test[:10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
예측 결과: [0 0 1 2 1 2 2 1 1 1]
실제 레이블: [0 0 1 2 1 2 2 1 1 1]


In [20]:
history = model.fit(
    x_train, y_train,
    epochs=30,
    batch_size=128,
    validation_split=0.1,
    verbose=2
)

Epoch 1/30
106/106 - 17s - 160ms/step - accuracy: 0.7730 - loss: 0.5334 - val_accuracy: 0.7313 - val_loss: 0.6105
Epoch 2/30
106/106 - 21s - 201ms/step - accuracy: 0.7764 - loss: 0.5169 - val_accuracy: 0.7127 - val_loss: 0.6527
Epoch 3/30
106/106 - 19s - 182ms/step - accuracy: 0.7873 - loss: 0.4943 - val_accuracy: 0.7520 - val_loss: 0.5804
Epoch 4/30
106/106 - 22s - 203ms/step - accuracy: 0.7997 - loss: 0.4687 - val_accuracy: 0.7513 - val_loss: 0.5890
Epoch 5/30
106/106 - 17s - 162ms/step - accuracy: 0.8153 - loss: 0.4415 - val_accuracy: 0.7633 - val_loss: 0.5818
Epoch 6/30
106/106 - 17s - 158ms/step - accuracy: 0.8216 - loss: 0.4274 - val_accuracy: 0.7607 - val_loss: 0.5832
Epoch 7/30
106/106 - 20s - 193ms/step - accuracy: 0.8285 - loss: 0.4119 - val_accuracy: 0.7580 - val_loss: 0.5875
Epoch 8/30
106/106 - 21s - 194ms/step - accuracy: 0.8441 - loss: 0.3800 - val_accuracy: 0.7707 - val_loss: 0.5951
Epoch 9/30
106/106 - 17s - 157ms/step - accuracy: 0.8487 - loss: 0.3685 - val_accuracy: 

In [21]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

94/94 - 1s - 13ms/step - accuracy: 0.7700 - loss: 0.7908

✅ Test Accuracy: 0.7700


In [23]:
predictions = model.predict(x_test[:10])
print("예측 결과:", np.argmax(predictions, axis=1))
print("실제 레이블:", y_test[:10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
예측 결과: [0 0 1 2 1 2 0 1 1 1]
실제 레이블: [0 0 1 2 1 2 2 1 1 1]
